In [26]:
import pandas as pd
import io
import re
import csv
import json
import numpy as np
from tqdm import tqdm
import math
import ast
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import seaborn as sns
import keras
from keras import backend as K
from progressbar import ProgressBar
import matplotlib.pyplot as plt
from keras import Sequential, Model
from random import randrange
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from keras.utils.vis_utils import plot_model
import time
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance


pbar = ProgressBar()
tqdm.pandas()

In [ ]:

path_processed_val_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/val_7.csv'
path_raw_val = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val_all.csv'
val = pd.read_csv(path_processed_val_nn,converters={'NN_dataframe': from_np_array})
def add_zero_blocks(arr):
    zero_blocks = np.zeros((max_blocks-arr.shape[0],arr.shape[1]))
    return np.append(arr, zero_blocks, axis=0)

val['NN_dataframe']= val['NN_dataframe'].apply(lambda x : add_zero_blocks(x))
val_hw = pd.read_csv(path_raw_val)
y_val = np.array(val_hw["total_power"].tolist())
y_val=y_val.reshape(y_val.shape[0],-1)
X_val_hw_param  = val_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
X_val_hw_param['tot_mac'] = X_val_hw_param['mac_num']*X_val_hw_param['mac_array_num']

In [27]:
max_blocks = 37
nb_param =7
nb_hw_param = 6

In [30]:

 model_nn= tf.keras.Sequential([
       layers.Conv1D(128, (3), activation='relu', padding="same",input_shape=(max_blocks, nb_param)),
       layers.Conv1D(64, (3), activation='relu', padding="same"),
       layers.Lambda( lambda x: K.sum(x, axis=1)),
      #layers.Flatten()
 ])


model_hw = tf.keras.Sequential([
     layers.Dense(128, activation='relu', input_shape=(nb_hw_param,)),
     #layers.Dense(64, activation='relu'),
     layers.Dense(64, activation='linear')
 ])


concat = tf.keras.layers.multiply([model_nn.output, model_hw.output])
output = tf.keras.layers.Dense(units=16, activation='relu')(concat)
output = tf.keras.layers.Dense(units=1, activation='relu')(output)


full_model = tf.keras.Model(inputs=[model_nn.input, model_hw.input], outputs=[output])



In [31]:
full_model.load_weights("/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/results_best_models/multiply/model_7_37665_param_0.213_error/model_7_37665_param")

In [33]:
def score(X_list,y):
    losses=[]
    inputs =[]
    for i in range(len(X_list[0])):
        for x in range(len(X_list)): # whether one or multiple inputs
            input_x=np.array(X_list[x])[i]
            input_x=input_x.reshape((1,*input_x.shape))
            inputs.append(tf.convert_to_tensor(input_x))
        pred = model.predict(inputs)
        losses.append(loss(y[i],pred))
    return losses

In [35]:
score(X_val_list[:10],y_val[:10])

NameError: name 'X_val_list' is not defined

In [ ]:
base_score, score_decreases = get_score_importances(score, X_val_list, y_val)

In [ ]:
with tf.io.TFRecordWriter('tmp.tfrecord') as writer:
    def create_float_feature(values):
        return tf.train.Feature(float_list=tf.train.FloatList(value=values))

    def _bytes_feature(value):
        """Returns a bytes_list from a string / byte."""
        if isinstance(value, type(tf.constant(0))): # if value ist tensor
            value = value.numpy() # get value of tensor
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


    def serialize_array(array):
        array = tf.io.serialize_tensor(array)
        return array


    def create_int_feature(values):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))

    for X_nn, X_hw,y in zip(X_val_nn_norm, X_val_hw_param_norm, y_val):
        serialized_array = serialize_array(X_nn)
        features = {'b_features':  _bytes_feature(serialized_array) , 'labels': create_float_feature([y])}
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())

def _parse_tfr_element(element):
  parse_dic = {
     'b_feature': tf.io.FixedLenFeature([], tf.string), # Note that it is tf.string, not tf.float32
     'labels': tf.FixedLenFeature([], tf.float64),
    }
  example_message = tf.io.parse_single_example(element, parse_dic)

  b_feature = example_message['b_feature'] # get byte string
  label = example_message['labels']
  feature = tf.io.parse_tensor(b_feature, out_type=tf.float64) # restore 2D array from byte string
  return feature, label


tfr_dataset = tf.data.TFRecordDataset('tmp.tfrecord') 
for serialized_instance in tfr_dataset:
  print(serialized_instance) # print serialized example messages

dataset = tfr_dataset.map(_parse_tfr_element)
for instance in dataset:
  print()
  print(instance) # print parsed example messages with restored arrays
  breakwith tf.io.TFRecordWriter('tmp.tfrecord') as writer:
    def create_float_feature(values):
        return tf.train.Feature(float_list=tf.train.FloatList(value=values))

    def _bytes_feature(value):
        """Returns a bytes_list from a string / byte."""
        if isinstance(value, type(tf.constant(0))): # if value ist tensor
            value = value.numpy() # get value of tensor
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


    def serialize_array(array):
        array = tf.io.serialize_tensor(array)
        return array


    def create_int_feature(values):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))

    for X_nn, X_hw,y in zip(X_val_nn_norm, X_val_hw_param_norm, y_val):
        serialized_array = serialize_array(X_nn)
        features = {'b_features':  _bytes_feature(serialized_array) , 'labels': create_float_feature([y])}
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())

def _parse_tfr_element(element):
  parse_dic = {
     'b_feature': tf.io.FixedLenFeature([], tf.string), # Note that it is tf.string, not tf.float32
     'labels': tf.FixedLenFeature([], tf.float64),
    }
  example_message = tf.io.parse_single_example(element, parse_dic)

  b_feature = example_message['b_feature'] # get byte string
  label = example_message['labels']
  feature = tf.io.parse_tensor(b_feature, out_type=tf.float64) # restore 2D array from byte string
  return feature, label


tfr_dataset = tf.data.TFRecordDataset('tmp.tfrecord') 
for serialized_instance in tfr_dataset:
  print(serialized_instance) # print serialized example messages

dataset = tfr_dataset.map(_parse_tfr_element)
for instance in dataset:
  print()
  print(instance) # print parsed example messages with restored arrays
  break